# Twoja własna wirtualna kamera - czyli sieci neuronowe dla każdego.
Abstrakt: *Naucz się tworzyć swoje własne doświadczenia ze Sztuczną Inteligencją. Na warsztacie spojrzymy na obecny krajobraz SI z czysto pragmatycznego punktu widzenia, oraz pokażemy jak używać gotowych modeli w celu tworzenia nowych produktów, niezwłocznie przechodząc do budowy Waszego pierwszego narzędzia. Wirtualnej kamery nafaszerowanej sieciami neuronowymi.*

## Wykrywanie twarzy

W pierwszej kolejności, spróbujmy wykryć twarz na różne sposoby.

### Instalacja zależności i importy
`!` pozwala na uruchomienie linii komend i zainstalowanie zależności wprost z notatnika. Alternatywą byłoby dołączenie `requirements.txt`, ale zgodnie z zasadą *Reproducible Research*, starajmy się pisać notatniki tak, aby były w jak największym stopniu samowystarczalne.

In [ ]:
!pip install numpy opencv-python matplotlib

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline

### Pobranie obrazu z kamerki
Przy sporej większości przypadków związanych z *Computer Vision* używa się OpenCV. Nie inaczej postąpimy tutaj.

In [ ]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise "Nie można otworzyć kamerki"
ret, frame = cap.read()
plt.imshow(frame)

Obraz w OpenCV jest domyślnie trzymany jako BGR, czyli opisuje go kolejno Niebieski, Zielony i Czerwony, w przeciwieństwie do RGB powszechnego w grafice komputerowej. Powód, jak zazwyczaj, jest historyczny. Format BGR był popularny w swoim czasie wśród producentów kamer oraz oprogramowania do obróbki obrazu.

In [ ]:
plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

### Rozpoznawanie twarzy

#### Kaskady Haar-a

Nazwa wzięła się od tego że jądro splotu (*Convolution Kernel*) w algorytmie przypomina [falki Haar'a](https://en.wikipedia.org/wiki/Haar_wavelet) właśnie. Algorytm opisuje praca [*Rapid Object Detection using a Boosted Cascade of Simple Features*](https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf) z 2001. Sposób działania algorytmu możecie prześledzić na wideo poniżej:

In [ ]:
%%HTML
<iframe src="https://player.vimeo.com/video/12774628?title=0&byline=0&portrait=0" width="700" height="394" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen></iframe>

Kaskady Haar'a są wbudowane w OpenCV. To jest pierwszy, i w oczywiście najprostszy, sposób w jaki możemy używać *Machine Learning*. Sporo bibliotek ma popularne funkcjonalności już wbudowane. Najważniejsze w tym wypadku jest doczytać dokumentację, jakich danych model oczekuje na wejściu. W tym przypadku, musimy mieć obrazek w odcieniach szarości, o wyrównanym histogramie.

Pliki modelu w tym wypadku są rozprowadzane z OpenCV, ale możemy je też pobrać bezpośrednio ze [strony](https://github.com/opencv/opencv/tree/master/data/haarcascades) i wersjonować lokalnie.

In [ ]:
haar_img = frame.copy()
haar_img_gray = cv.cvtColor(haar_img, cv.COLOR_BGR2GRAY)
haar_img_gray = cv.equalizeHist(haar_img_gray)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
faces = face_cascade.detectMultiScale(haar_img_gray)
for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)
        haar_img = cv.rectangle(haar_img, (x, y), (x+w, y+h), (255, 0, 0), 2)
plt.imshow(cv.cvtColor(haar_img, cv.COLOR_BGR2RGB))

Kaskady Haar'a możemy wykorzystywać nie tylko do wykrywania twarzy. Są też modele do wykrywania oczu, części ciała, czy nawet rosyjskich tablic rejestracyjnych... 

In [ ]:
eyes_cascade = cv.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
for (x,y,w,h) in faces:
    faceROI = haar_img_gray[y:y+h,x:x+w]
    eyes = eyes_cascade.detectMultiScale(faceROI)
    for (x2,y2,w2,h2) in eyes:
        eye_center = (x + x2 + w2//2, y + y2 + h2//2)
        radius = int(round((w2 + h2)*0.25))
        haar_img = cv.circle(haar_img, eye_center, radius, (255, 0, 0 ), 4)
plt.imshow(cv.cvtColor(haar_img, cv.COLOR_BGR2RGB))

### Dlib

Dlib jest biblioteką algorytmów uczenia maszynowego, napisaną w C++. Posiada natomiast wiązania do pythona. Jest to bardzo popularne rozwiązanie: kod implementujący algorytmy, którego wydajność jest kluczowa, pisany jest w C/C++, CUDA czy częściowo w assemblerze, a API wystawiane jest do języka wyższego poziomu za pomocą tzw. *bindings*.

Dlib posiada algorytm wykrywania twarzy oparty o konwolucyjne sieci neuronowe (*Convolutional Neural Networks*), ale nie nadaje się on do pracy w czasie rzeczywistym, dlatego przyjrzymy się innemu - wykrywaniu twarzy za pomocą Histogramu Gradientu Kierunkowego czyli *Histogram of Oriented Gradients*. Algorytm po raz pierwszy został opisany w [patencie](https://patents.google.com/patent/US4567610) z 1986, ale metoda nie zyskała popularności do czasu publikacji pracy [*Histograms of Oriented Gradients for Human Detection*](http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf) w 2005.

In [ ]:
!pip install dlib

In [ ]:
import dlib

In [ ]:
dlib_img = frame.copy()
detector = dlib.get_frontal_face_detector()
dlib_img_gray = cv.cvtColor(dlib_img, cv.COLOR_BGR2GRAY)
faces = detector(dlib_img_gray, 1) 
for result in faces:
    x = result.left()
    y = result.top()
    x1 = result.right()
    y1 = result.bottom()
    dlib_img = cv.rectangle(dlib_img, (x, y), (x1, y1), (0, 0, 255), 2)
plt.imshow(cv.cvtColor(dlib_img, cv.COLOR_BGR2RGB))

## Multi-task Cascaded Convolutional Networks

MTCCN opublikował po raz pierwszy Kaipeng Zhang wraz z innymi, w artykule z 2016 pt. [“Joint Face Detection and Alignment Using Multi-task Cascaded Convolutional Networks”](https://arxiv.org/abs/1604.02878). Wykyrwa on nie tylko twarze, ale także 5 kluczowych punktów tzw. *facial landmarks*.

In [ ]:
from mtcnn.mtcnn import MTCNN

In [ ]:
mtcnn_img = frame.copy()
detector = MTCNN()
faces = detector.detect_faces(mtcnn_img)
for result in faces:
    x, y, w, h = result['box']
    x1, y1 = x + w, y + h
    mtcnn_img = cv.rectangle(mtcnn_img, (x, y), (x1, y1), (0, 0, 255), 2)
plt.imshow(cv.cvtColor(mtcnn_img, cv.COLOR_BGR2RGB))

### OpenCV DNN

Stosunkowo niedawno OpenCV otrzymał swój własny moduł głębokiego uczenia. Dołączony model rozpoznawania twarzy, największą dokładność ma na obrazach w formacie BGR, pomniejszonych do rozmiaru 300x300 pikseli.

Wszystkie modele dostępne w module DNN opisane są [tu](https://github.com/opencv/opencv/blob/master/samples/dnn/models.yml).
    
My będziemy używać 10 warstwowego modelu ResNET, z [opisem](https://github.com/opencv/opencv/blob/master/samples/dnn/face_detector/deploy.prototxt) i [wagami](https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel).

In [ ]:
dnn_img = frame.copy()
net = cv.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")
h, w = dnn_img.shape[:2]
blob = cv.dnn.blobFromImage(cv.resize(dnn_img, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
net.setInput(blob)
faces = net.forward()
for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            dnn_img = cv.rectangle(dnn_img, (x, y), (x1, y1), (0, 0, 255), 2)
plt.imshow(cv.cvtColor(dnn_img, cv.COLOR_BGR2RGB))

### BodyPix

W 2019 roku, Google [wypuściło w świat](https://medium.com/tensorflow/introducing-bodypix-real-time-person-segmentation-in-the-browser-with-tensorflow-js-f1948126c2a0) swój model BodyPix. Nie dość, że potrafi on działać w czasie rzeczywistym, osiągając powyżej 20fps nawet na starszych sprzętach, to wykrywa nie tylko twarz, ale segmentuje całe ciało na aż 24 części!

Używa albo MobileNetV1 albo ResNet50. Różnica między nimi jest głównie w rozmiarze, a co za tym idzie, wymaganiach sprzętowych i szybkości działania.


| Architecture | quantBytes=4 | quantBytes=2 | quantBytes=1 |
|--------------|:------------:|:------------:|:------------:|
|ResNet50 | ~90MB | ~45MB | ~22MB|
|MobileNetV1 (1.00) | ~13MB | ~6MB | ~3MB|
|MobileNetV1 (0.75) | ~5MB | ~2MB | ~1MB|
|MobileNetV1 (0.50) | ~2MB | ~1MB | ~0.6MB|

Model do ściągnięcia [tu](https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/float/050/model-stride16.json), a wagi [tu](https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/float/050/group1-shard1of1.bin).


In [ ]:
!pip install tf_bodypix[all]==0.3.5 tensorflow-gpu==2.4.1 tfjs_graph_converter

In [ ]:
import tensorflow as tf
from tf_bodypix.api import load_model

In [ ]:
bodypix_model = load_model('mobilenet-float-50-model-stride16.json')
bodypix_img = frame.copy()
result = bodypix_model.predict_single(bodypix_img)
mask = result.get_mask(threshold=0.5).numpy().astype(np.uint8)
masked_image = result.get_colored_part_mask(mask, part_names=['left_face', 'right_face']).astype(np.uint8)
final = cv.addWeighted(bodypix_img, 0.6, masked_image, 0.4, 2.0)
plt.imshow(final)

In [ ]:
contours, hierarchy = cv.findContours(cv.cvtColor(masked_image, cv.COLOR_RGB2GRAY), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
for contour in contours:
    box = cv.boundingRect(contour)
    (x, y, w, h) = box
    final = cv.rectangle(final, (x, y), (x+w, y+h), (0, 0, 255), 2)
plt.imshow(final)

#### Podsumowanie

Kaskady Haara są dość przestarzałe i, gdyby zrobić takie porządne porównanie, z różnorodnym zbiorem testowym, to dawałyby najgorsze rezultaty. Dlib nie wykrywa twarzy na obrazkach mniejszych niż 80x80. Można je oczywiście powiększyć, ale to pogorszy jednocześnie wydajność, która nie jest mocną stroną Dliba i raczej nie nadaje się do pracy w czasie rzeczywistym poza komputerami stacjonarnymi. Moduł DNN z OpenCV i MTCNN radzą sobie podobnie, choć ten ostatni lepiej wypada gdy obrazy są duże.

BodyPix wypada równie dobrze, a przy okazji oferuje dużo więcej informacji.

## Wirtualne tło

Jednym z przykładów, gdzie ta większa ilość informacji oferowana przez BodyPix może się przydać, to wirtualne tło, albo tzw. *virtual greenscreen*. Mając informację o korpusie postaci i twarzy, możemy jej użyć do usunięcia wszystkiego co znajduje się poza.

W tym celu, załadujmy najpierw obraz, który będzie stanowił nasze tło.

In [ ]:
(fh, fw, fc) = frame.shape

background = cv.imread('wawel-noca.jpg')
(bh, bw, bc) = background.shape

dif = bw if bh > bw else bh
fdif = fw if fh > fw else fh

x_pos = (bw - dif)//2
y_pos = (bh - dif)//2

background_mask = np.zeros((dif, dif, bc), dtype=background.dtype)
background_mask[:dif, :dif, :] = background[y_pos:y_pos+dif, x_pos:x_pos+dif, :]

background_squared = cv.resize(background_mask, (int(fdif*2), int(fdif*2)), cv.INTER_CUBIC)


plt.imshow(cv.cvtColor(background_squared, cv.COLOR_BGR2RGB))

Następnie pobierzemy maskę z BodyPix'a. Maska będzie miała `0` tam gdzie nie ma postaci, oraz `1` tam gdzie BodyPix postać wykrył. Możemy jej użyć do wycięcia postaci z naszej ramki z kamery. Do wymaskowania tła, będziemy musieli wykonać negację maski, a następnie dodamy tło i ramkę do siebie.

In [ ]:
(bh, bw, bc) = background_squared.shape

x_pos = (bw - fw)//2
y_pos = (bh - fh)//2

background = background_squared[y_pos:y_pos+fh, x_pos:x_pos+fw,:]

mask = result.get_mask(threshold=0.5).numpy().astype(np.uint8)
masked_image = cv.bitwise_and(frame, frame, mask=mask)
neg = np.add(mask, -1)
inverse = np.where(neg == -1, 1, neg).astype(np.uint8)
masked_background = cv.bitwise_and(background, background, mask=inverse)
final = cv.add(masked_image, masked_background)
plt.imshow(cv.cvtColor(final, cv.COLOR_BGR2RGB))

## Żywa kamera

Spróbujmy teraz zrobić to samo, ale zamiast pracować na pojedynczym ujęciu z kamery, nałóżmy to na podgląd na żywo.

In [ ]:
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    result = bodypix_model.predict_single(frame)
    (bh, bw, bc) = background_squared.shape

    x_pos = (bw - fw)//2
    y_pos = (bh - fh)//2

    background = background_squared[y_pos:y_pos+fh, x_pos:x_pos+fw,:]

    mask = result.get_mask(threshold=0.5).numpy().astype(np.uint8)
    masked_image = cv.bitwise_and(frame, frame, mask=mask)
    neg = np.add(mask, -1)
    inverse = np.where(neg == -1, 1, neg).astype(np.uint8)
    masked_background = cv.bitwise_and(background, background, mask=inverse)
    final = cv.add(masked_image, masked_background)
    
    cv.imshow("Kamera", final)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

## Paralaksa

Kolejną rzeczą którą chciałbym zaproponować, jest efekt tzw. paralaksy. W skrócie polega on na ożywieniu tła za kamerą, tak, że wygląda ono jak trójwymiarowe. Przesuwając twarz, jakby oglądamy je z pod trochę innym kątem. 

W tym celu, będziemy śledzić twarz i jej pozycję. Do śledzenia, wybierzemy twarz o największej obwiedni. W rzeczywistości, algorym powinien być bardziej złożony. Przydatne byłoby choćby uchwycenie twarzy, aby w wypadku gdy aktualnie śledzona twarz oddali się i pojawi się kandydat o większej obwiedni, mimo wszystko śledzić dalej poprzednią, unikając widocznego przeskoku.

Dla uchwyconej twarzy, obliczymy jej odchylenie od środka obrazu, żeby odpowiednio przesunąć tło.


In [ ]:
import functools

cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    result = bodypix_model.predict_single(frame)
    (bh, bw, bc) = background_squared.shape

    mask = result.get_mask(threshold=0.5).numpy().astype(np.uint8)
    face_masked_image = result.get_colored_part_mask(mask, part_names=['left_face', 'right_face']).astype(np.uint8)
    contours, hierarchy = cv.findContours(cv.cvtColor(face_masked_image, cv.COLOR_RGB2GRAY), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    face = functools.reduce(lambda a,b: a if a[2]+a[3] > b[2]+b[3] else b, list(map(lambda a: cv.boundingRect(a), contours)))
    
    x_pos = (bw - fw)//2
    y_pos = (bh - fh)//2
    
    if face:
        (x, y, w, h) = face
        face_x = x+w//2
        face_y = y+h//2
        x_pos -= int((fw-face_x)*0.25)
        y_pos -= int((fh-face_y)*0.25)

    background = background_squared[y_pos:y_pos+fh, x_pos:x_pos+fw,:]
    
    masked_image = cv.bitwise_and(frame, frame, mask=mask)
        
    neg = np.add(mask, -1)
    inverse = np.where(neg == -1, 1, neg).astype(np.uint8)
    masked_background = cv.bitwise_and(background, background, mask=inverse)
    final = cv.add(masked_image, masked_background)
    if face:
        (x, y, w, h) = face
        final = cv.rectangle(final, (x, y), (x+w, y+h), (0, 0, 255), 2)
        final = cv.circle(final, (x+w//2, y+h//2), 5, (255, 0, 0 ), 4)
            
    cv.imshow("Kamera", final)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

## Podsumowanie

Pierwszą rzeczą którą zuważyliście zapewne uruchamiając przykład jest fakt, że obwiednia twarzy, a co za tym idzie wyliczony środek ciężkości, co ramkę zmienia delikatnie rozmiar. Wpływa to niestety na płynność przesuwania tła. Następnym krokiem, mogłoby być policzenie ważonej średniej ruchomej.

Co jeszcze dalej? Mając powyższe modele, oraz moduł `pyvirtualcam`, możecie choćby spróbować sami zaimplementować swoją własną [Snap Kamerę](https://snapcamera.snapchat.com/).

Owocnej zabawy!

W razie pytań, nie obawiajcie się zagadnąć :) Znaleźć możecie mnie tu:

- https://twitter.com/unjello
- http://poly.work/unjello
- https://www.linkedin.com/in/andrzejlichnerowicz/